# Scraping b-ok site (name of books , urls) using selenium , then download all books 
This part is used to download pdfs from b-ok website.
Unfortunately, i could only download only 5 book cause we must have an account on the website.
if we have an account we can download 10 books but i couldn't sign in while scrapping

In [1]:
#Importing packages
!pip install selenium
from selenium import webdriver
import pandas as pd
import os 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
#from fake_useragent import UserAgent
import pandas as pd
from collections.abc import Iterable
import warnings
# from pymongo import MongoClient
# from collections.abc import Iterable
import time

In [2]:
selenium_driver_path = "C:/Users/SMSM-TECH/Downloads/chromedriver_win32/chromedriver.exe"

In [3]:
# get unique elements from a list
def unique(data):
    return list(dict.fromkeys(data))


# get selenium driver object
def get_selenium_driver():
    """
    This function returns the selenium driver object.

    Parameters:
        None

    Returns:
        driver: selenium driver object
    """

    driver = webdriver.Chrome(executable_path = selenium_driver_path)

    return driver


# get BeautifulSoup object
def get_soup(url):
    """
    Given the url of a page, this function returns the soup object.
    
    Parameters:
        url: the link to get soup object for
    
    Returns:
        soup: soup object
    """
    driver = get_selenium_driver()

    driver.get(url)

    driver.implicitly_wait(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    driver.close()

    return soup

#save to MongoDB
def saveToDB(collection, city_areas_zones_dict):
    
    try:
        print("collection.count_documents({}): ",  collection.count_documents({}))
        resultSet = collection.count_documents({"City":city_areas_zones_dict["City"],"Areas_Zones":city_areas_zones_dict['Areas_Zones']})

        if resultSet > 0:
            print("city_areas_zones_dict is already scrapped >> ", city_areas_zones_dict)
            print("Skip!")
        else:
            print("New city_areas_zones_dict will be scrapped >> ", city_areas_zones_dict)
            print("I will insert it!")
            collection.insert_one(city_areas_zones_dict)
            
    except Exception as e:
        print('Exception : %s' % str(e))

In [4]:
import re

# scrape b-ok website

def scrape_books():
    books_names = []

    for num in range(1,2):
        base_url = "https://b-ok.cc/s/natural%20language%20processing?page="+str(num)

        home_page_soup = get_soup(base_url)

        # get a get_selenium_driver object
        driver = get_selenium_driver()
        #driver.execute_script("return navigator.userAgent")
        driver.get(base_url)
        

        home_page_soup=home_page_soup.find_all("div",{'id':"searchResultBox"})
        links=home_page_soup[0].find_all("a",{"href":re.compile("^/book/")})
        #print(links)
            

        all_link = []
        for link in links:
            part_of_link = link.get("href")
            link = "https://b-ok.cc"+part_of_link
            all_link.append(link)
            all_link = list(set(all_link))
            print("all_link")
            print(len(all_link))

            print(all_link)
        
        for link in all_link:
            page_driver2 = get_selenium_driver()
            page_driver2.get(link)
            page_driver2.find_element_by_class_name("dlButton").click()
            


        
        time.sleep(3)
        print("----------------------------------")
    driver.close()
        


In [5]:
scrape_books()

all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3681492/07d64c']
all_link
5
['https://b-ok.cc/book/755716/be52d7', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3711505/debac

ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" id="dbx_3362289" class="btn btn-primary dlButton disabled" data-toggle="tooltip" data-autoopen="true" data-placement="top" title="" data-original-title="Link deleted by legal owner (But you can use <a href=&quot;/msgn/tor&quot; target=&quot;_blank&quot; rel=&quot;nofollow&quot;>TOR</a> to download it anyway)" aria-describedby="tooltip105214">...</a> is not clickable at point (142, 604). Other element would receive the click: <div class="book-details-button">...</div>
  (Session info: chrome=80.0.3987.149)


# Table Of Content Summarization Steps
1. Reading the table of content.
2. Choosing only level one sentences to summarize.
3. Cleaning TOC sentences.
4. Summarizing each sentence in TOC:
    1. Splitting into words (Tokenization).
    2. Vectorizing each word using TF-Hub Universal sentence Encoder (embedding matrix).
    3. Applaying Kmeans clusturing to cluster the words into certain no. of clusters.
    4. Forming sentence summary from the words which are the nearst to clusteres centroids. from each cluster we take a words this word is the closest to the centroid. Then we join these words together to form the summarized sentence.
5. Printing the summary of TOC.

###### note: this notebook needs to be ran on colab as it uses the GPU version of tensorflow or you need to :
  1. Install the latest Tensorflow version.
     - !pip3 install --upgrade tensorflow-gpu
  2. Install TF-Hub by
     - !pip3 install tensorflow-hub 
     - !pip3 install seaborn

In [1]:
#installing pdfminer
!pip install pdfminer

In [2]:
# Importing the required libraries
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import tensorflow_hub as hub
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Defining required functions for TOC reading 

In [0]:
def find_last_level(list_of_tuples,level):
    """" this function gets the last title of a certain level
    
     parameters:
         list_of_tuples: is a list of tubles each tuple consists of (level,title)
         level: is the level that we are tying to get the last title of from the list of tuples
         
     return:
         title: is the last title of the chosen level"""
    
    list_of_levels = []
    for i in list_of_tuples:
        if i[0] == level:
            list_of_levels.append(i)
    title = list_of_levels[-1][1]
    return title
        

In [0]:
def search_values(level,dct,main_level,last,title):
    """"this function uses recurssion to find the parent of a certain title
    
    parameters:
        level: is the level-1 of the title we are looking for its parent we use this parameter for recurssion
        dct: is the dictionry we are searching in
        main_level: is the same as level parameter but used to save the first value of level in the case of recurrsion
        last: list of tuples consists of [(level,title)]
        title: the main title we are looking for it's parent
        
    return:
        dct: the updated dictionary"""
    
    if find_last_level(last,level) in list(dct.keys()):
        if find_last_level(last,level) != find_last_level(last,main_level):
            if type(dct[find_last_level(last,level)][-1]) is dict:
                new_dct = dct[find_last_level(last,level)][-1]
                dct[find_last_level(last,level)][-1] = search_values(main_level,new_dct,main_level,last,title)
                return dct
            else:
                dct[find_last_level(last,level)][-1] = {str(dct[find_last_level(last,level)][-1]):[title]}
                return dct
        else:
            dct[find_last_level(last,level)].append(title)
            return dct
    else:
        return search_values(level-1,dct,main_level,last,title)
            

### Loading TF unversal sentence embedding pretrained model & defining the embedding function

In [5]:
   
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


### Defining stop words list for cleaning
note: I removed certain words from nltk stop words i felt that they may affect the sentence meaning.

In [0]:
stop = stopwords.words('english')
stop2 = [ 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under','not']
stop=list(set(stop) - set(stop2))

## Summarizing TOC of the first PDF

### Reading Table of Content of the PDF

In [7]:
# Open a PDF document.
fp = open('drive/My Drive/Colab/task6/Neural Network Methods for Natural Language Processing by Yoav Goldberg (z-lib.org).pdf', 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)
dct = {}
last = []             
# Get the outlines of the document (entire table of content).
outlines = document.get_outlines()
for (level,title,dest,a,se) in outlines:
    if level == 1:
        last.append((level,title))
        dct[title] = []
    elif level > 1:
        last.append((level,title))
        search_values(level-1,dct,level-1,last,title)

# Getting only level 1 headings in TOC
headings_one=list(dct.keys())
print("Table of Content (Level one only):\n")
for i in range(len(headings_one)):
  print("  ",headings_one[i])

Table of Content (Level one only):

   Contents
   Preface
   Introduction
   Supervised Classification & Feed-Forward NNs
   Learning Basics & Linear Models
   From Linear Models to Multi-layer Perceptrons
   Feed-Forward NNs
   Neural Network Training
   --- Natural Language Data
   Features for Textual Data
   Case Studies of NLP Features
   From Textual Features to Inputs
   Language Modeling
   Pre-trained Word Representations
   Word Embeddings
   Case Study - Feed-Forward Architecture for Sentence Meaning Inference
   --- Specialized Architectures
   Ngram Detectors - Convolutional NNs
   Recurrent NNs - Modeling Sequences & Stacks
   Concrete Recurrent NN Architectures
   Modeling with Recurrent Networks
   Conditioned Generation
   --- Additional Topics
   Modeling Trees with Recursive NNs
   Structured Output Prediction
   Cascaded, Multi-task & Semi-supervised Learning
   Conclusion
   Biblio


### Cleaning table of content

In [8]:
cleaned_headings=[]
for i in range(len(headings_one)):
    x= headings_one[i]
    # removing puctuations
    x= re.sub('[^\w\s]', '', x ).strip()
    # replacing some abbreviations
    x= re.sub('NNs', 'Neural Networks', x ).strip()
    x= re.sub('NN', 'Neural Network', x ).strip()
    # removing stop words
    x= ' '.join( [word for word in x.split() if word.lower().strip() not in stop])
    cleaned_headings.append(x)
# Printing after Cleaning
print("Table of Content after Cleaning:\n")
for i in range(len(cleaned_headings)):
  print("  ",cleaned_headings[i])

Table of Content after Cleaning:

   Contents
   Preface
   Introduction
   Supervised Classification FeedForward Neural Networks
   Learning Basics Linear Models
   From Linear Models to Multilayer Perceptrons
   FeedForward Neural Networks
   Neural Network Training
   Natural Language Data
   Features for Textual Data
   Case Studies of NLP Features
   From Textual Features to Inputs
   Language Modeling
   Pretrained Word Representations
   Word Embeddings
   Case Study FeedForward Architecture for Sentence Meaning Inference
   Specialized Architectures
   Ngram Detectors Convolutional Neural Networks
   Recurrent Neural Networks Modeling Sequences Stacks
   Concrete Recurrent Neural Network Architectures
   Modeling with Recurrent Networks
   Conditioned Generation
   Additional Topics
   Modeling Trees with Recursive Neural Networks
   Structured Output Prediction
   Cascaded Multitask Semisupervised Learning
   Conclusion
   Biblio


### Defining a function to summerize sentences

In [0]:
def Summarize_sentence(sentence):
  # printing the sent. before summarization
  print("Before:",sentence)
  # converting into list of words(tokenization)
  sentence=sentence.split()
  # embedding all the words in the list(vectorization)
  sen_vectors= embed(sentence)
  # from tensors to list of vectors (embedding matrix)
  sen_vectors= np.array(sen_vectors).tolist()
  # clustering using kmeans but only for sentences that have length larger than 3 
  if len(sentence) > 3 :
    # defining the number of clusters as the ceil of 2/3 from the original length of the sentence
    n_clusters = int(np.ceil(len(sentence)*2/3))
    kmeans = KMeans(n_clusters=n_clusters, init = 'k-means++', random_state = 42)
    kmeans = kmeans.fit(sen_vectors)
    # obtaining the indices of the nearst words to clusters centroids
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sen_vectors)
    # sorting the indices
    closest.sort()
    # joining the words to obtain the summary
    summary=[]
    for i in closest:
      summary.append(sentence[i])
    summary=' '.join(summary)
  else:
    # if the lenght < 3 the summary is the same as the original sentence
    summary = ' '.join(sentence)
  # printing the summary
  print("After:",summary,'\n')
  return summary

### Obtaining the summarized TOC

In [10]:
summaries=[]
for sen in cleaned_headings:
  summaries.append(Summarize_sentence(sen))


Before: Contents
After: Contents 

Before: Preface
After: Preface 

Before: Introduction
After: Introduction 

Before: Supervised Classification FeedForward Neural Networks
After: Supervised Classification FeedForward Networks 

Before: Learning Basics Linear Models
After: Learning Linear Models 

Before: From Linear Models to Multilayer Perceptrons
After: Linear Models to Perceptrons 

Before: FeedForward Neural Networks
After: FeedForward Neural Networks 

Before: Neural Network Training
After: Neural Network Training 

Before: Natural Language Data
After: Natural Language Data 

Before: Features for Textual Data
After: Features for Textual 

Before: Case Studies of NLP Features
After: Studies of NLP Features 

Before: From Textual Features to Inputs
After: Textual Features to Inputs 

Before: Language Modeling
After: Language Modeling 

Before: Pretrained Word Representations
After: Pretrained Word Representations 

Before: Word Embeddings
After: Word Embeddings 

Before: Case Study

### Table of Content after Summarization

In [11]:
print("Table of Content after Summarization:\n")
for i in range(len(summaries)):
  print("  ",summaries[i])

Table of Content after Summarization:

   Contents
   Preface
   Introduction
   Supervised Classification FeedForward Networks
   Learning Linear Models
   Linear Models to Perceptrons
   FeedForward Neural Networks
   Neural Network Training
   Natural Language Data
   Features for Textual
   Studies of NLP Features
   Textual Features to Inputs
   Language Modeling
   Pretrained Word Representations
   Word Embeddings
   Case Study Architecture for Sentence Inference
   Specialized Architectures
   Ngram Detectors Convolutional Networks
   Recurrent Neural Networks Stacks
   Concrete Recurrent Network Architectures
   with Recurrent Networks
   Conditioned Generation
   Additional Topics
   Trees with Recursive Networks
   Structured Output Prediction
   Cascaded Multitask Learning
   Conclusion
   Biblio


## Summarizing TOC of the second PDF

### Reading Table of Content of the PDF

In [12]:
# Open a PDF document.
fp = open('drive/My Drive/Colab/task6/Power excel 2016 with mrexcel  Master Pivot Tables, Subtotals, Charts, VLOOKUP, IF, Data Analysis in Excel 2010–2013 by Bill Jelen (z-lib.org).pdf', 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)
dct = {}
last = []             
# Get the outlines of the document (entire table of content).
outlines = document.get_outlines()
for (level,title,dest,a,se) in outlines:
    if level == 1:
        last.append((level,title))
        dct[title] = []
    elif level > 1:
        last.append((level,title))
        search_values(level-1,dct,level-1,last,title)

# Getting only level 1 headings in TOC
headings_one=list(dct.keys())
print("Table of Content (Level one only):\n")
for i in range(len(headings_one)):
  print("  ",headings_one[i])

Table of Content (Level one only):

   Cover
   Title Page
   Credits
   Table of Contents
   About the Author
   Dedication
   Foreword
   Acknowledgments
   Why Does Office 365 Have Better Features?
   Which Version of Office 365 Has Power Pivot?
   Why Do I Have to Sign in to Excel?
   How Can I Use Excel on Dual Monitors?
   How Can I Open The Same Workbook Twice?
   Find Icons on the Ribbon
   Where is File, Exit?
   Where Are My Macros?
   Customizing the Ribbon
   Go Wide
   Minimize the Ribbon to Free Up a Few More Rows
   Use a Wheel Mouse to Scroll Through The Ribbon Tabs
   Why Do The Charting Ribbon Tabs Keep Disappearing?
   Use Dialog Launchers For More Choices
   Icon, Dropdowns, and Hybrids
   Zoom is at the Bottom
   Make Your Most-Used Icons Always Visible
   The Excel 2003 Alt Keystrokes Still Work
 (If You Type Them Slowly Enough)
   Use Keyboard Shortcuts to Access the Ribbon
   Why Do I Have Only 65,536 Rows?
   Which File Format Should I Use?
   Why Does The File

### Cleaning table of content

In [13]:
cleaned_headings=[]
for i in range(len(headings_one)):
    x= headings_one[i]
    # removing puctuations
    x= re.sub('[^\w\s]', '', x ).strip()
    # replacing some abbreviations
    x= re.sub('NNs', 'Neural Networks', x ).strip()
    x= re.sub('NN', 'Neural Network', x ).strip()
    # removing stop words
    x= ' '.join( [word for word in x.split() if word.lower().strip() not in stop])
    cleaned_headings.append(x)
# Printing after Cleaning
print("Table of Content after Cleaning:\n")
for i in range(len(cleaned_headings)):
  print("  ",cleaned_headings[i])

Table of Content after Cleaning:

   Cover
   Title Page
   Credits
   Table of Contents
   About Author
   Dedication
   Foreword
   Acknowledgments
   Office 365 Better Features
   Version of Office 365 Power Pivot
   to Sign in to Excel
   Use Excel on Dual Monitors
   Open Workbook Twice
   Find Icons on Ribbon
   File Exit
   Macros
   Customizing Ribbon
   Go Wide
   Minimize Ribbon to Free Up Rows
   Use Wheel Mouse to Scroll Through Ribbon Tabs
   Charting Ribbon Tabs Keep Disappearing
   Use Dialog Launchers For Choices
   Icon Dropdowns and Hybrids
   Zoom at Bottom
   Make MostUsed Icons Always Visible
   Excel 2003 Alt Keystrokes Still Work If Type Slowly Enough
   Use Keyboard Shortcuts to Access Ribbon
   65536 Rows
   File Format Use
   File Menu Cover Entire Screen
   Close File Menu
   Increase Number of Workbooks in Recent Files List
   Change Print Settings in Excel
   Want Old Print Preview Back
   Get Quick Access to Formatting Options Using Mini Toolbar
   Protect

### Defining a function to summerize sentences

In [0]:
def Summarize_sentence(sentence):
  # printing the sent. before summarization
  print("Before:",sentence)
  # converting into list of words(tokenization)
  sentence=sentence.split()
  # embedding all the words in the list(vectorization)
  sen_vectors= embed(sentence)
  # from tensors to list of vectors (embedding matrix)
  sen_vectors= np.array(sen_vectors).tolist()
  # clustering using kmeans but only for sentences that have length larger than 3 
  if len(sentence) > 3 :
    # defining the number of clusters as the ceil of 2/3 from the original length of the sentence
    n_clusters = int(np.ceil(len(sentence)*2/3))
    kmeans = KMeans(n_clusters=n_clusters, init = 'k-means++', random_state = 42)
    kmeans = kmeans.fit(sen_vectors)
    # obtaining the indices of the nearst words to clusters centroids
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sen_vectors)
    # sorting the indices
    closest.sort()
    # joining the words to obtain the summary
    summary=[]
    for i in closest:
      summary.append(sentence[i])
    summary=' '.join(summary)
  else:
    # if the lenght < 3 the summary is the same as the original sentence
    summary = ' '.join(sentence)
  # printing the summary
  print("After:",summary,'\n')
  return summary

### Obtaining the summarized TOC

In [15]:
summaries=[]
for sen in cleaned_headings:
  summaries.append(Summarize_sentence(sen))


Before: Cover
After: Cover 

Before: Title Page
After: Title Page 

Before: Credits
After: Credits 

Before: Table of Contents
After: Table of Contents 

Before: About Author
After: About Author 

Before: Dedication
After: Dedication 

Before: Foreword
After: Foreword 

Before: Acknowledgments
After: Acknowledgments 

Before: Office 365 Better Features
After: 365 Better Features 

Before: Version of Office 365 Power Pivot
After: Version 365 Power Pivot 

Before: to Sign in to Excel
After: to Sign in Excel 

Before: Use Excel on Dual Monitors
After: Excel on Dual Monitors 

Before: Open Workbook Twice
After: Open Workbook Twice 

Before: Find Icons on Ribbon
After: Find Icons Ribbon 

Before: File Exit
After: File Exit 

Before: Macros
After: Macros 

Before: Customizing Ribbon
After: Customizing Ribbon 

Before: Go Wide
After: Go Wide 

Before: Minimize Ribbon to Free Up Rows
After: Minimize Ribbon Free Up 

Before: Use Wheel Mouse to Scroll Through Ribbon Tabs
After: Use Wheel Mouse t

### Table of Content after Summarization

In [16]:
print("Table of Content after Summarization:\n")
for i in range(len(summaries)):
  print("  ",summaries[i])

Table of Content after Summarization:

   Cover
   Title Page
   Credits
   Table of Contents
   About Author
   Dedication
   Foreword
   Acknowledgments
   365 Better Features
   Version 365 Power Pivot
   to Sign in Excel
   Excel on Dual Monitors
   Open Workbook Twice
   Find Icons Ribbon
   File Exit
   Macros
   Customizing Ribbon
   Go Wide
   Minimize Ribbon Free Up
   Use Wheel Mouse to Ribbon Tabs
   Charting Ribbon Tabs Keep
   Use Dialog Launchers Choices
   Dropdowns and Hybrids
   Zoom at Bottom
   Make MostUsed Icons Visible
   Excel 2003 Alt Work Type Slowly Enough
   Use Keyboard Shortcuts Ribbon
   65536 Rows
   File Format Use
   File Menu Entire Screen
   Close File Menu
   Increase Number of Workbooks Recent Files
   Print Settings in Excel
   Want Print Preview Back
   Get Quick Formatting Options Mini Toolbar
   Protected Mode
   Trusted Location to Excels Constant Warnings
   Wants Create Expense Report from Scratch
   Open Copy of
   Excel with CtrlAltX
   Alw

## note that we can change the following:
1. the lenght after which we summerize the sentence (which i choose to be 3).
2. the number of clusters (here i choose to be 2/3 from the original length as it gave be good results).
3. the method of vectorization (here i choose TF universal sentence encoder as it gives good features but it can be any way of sentence embedding).
4. also, we notice that summerized sentences are not perfect (sometimes they lose the original meaning or they are not complete) but it was the best i can obtain using this method.